In [ ]:
# Copyright 2026 50Hertz Transmission GmbH and Elia Transmission Belgium SA/NV
#
# This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0.
# If a copy of the MPL was not distributed with this file,
# you can obtain one at https://mozilla.org/MPL/2.0/.
# Mozilla Public License, version 2.0

In [ ]:
from pathlib import Path

import jax
from toop_engine_dc_solver.preprocess.convert_to_jax import load_grid, run_initial_loadflow
from toop_engine_interfaces.messages.preprocess.preprocess_commands import PreprocessParameters
from fsspec.implementations.dirfs import DirFileSystem
# Load the function of the example grid
from toop_engine_dc_solver.example_grids import case57_data_powsybl
# Perform creation of data folder in 64 bits
jax.config.update("jax_enable_x64", True)

# Example 1: Compute a DC loadflow

In this example, we compute a DC loadflow using the [DC Solver package](./dc_solver/intro.md) on an example grid.

We import the `Powsybl` version of an IEEE57 grid and retrieve the necessary information to perform our task.  

## Load the grid

In [ ]:
folder = Path("../data/case57_data_powsybl") # Select folder to store data
case57_data_powsybl(folder) # Create the data for the IEEE 57 bus test case in PyPowsybl format

filesystem_dir = DirFileSystem(str(folder))
stats, static_information, network_data = load_grid( # Load the grid
    filesystem_dir,
    pandapower=False, # Use PyPowsybl backend
    parameters=PreprocessParameters( # Adjust parameters for preprocessing
        action_set_filter_bsdf_lodf = False,
        fail_on_non_convergence=False # Just an example on how to change parameters
    )
)

The `stats` object contains several statistics about the import and `static_information` includes all the required information for the Topology Optimizer to work.
The `network_data` object includes information necessary to later convert the results of the topological optimization into a backend format.

Let's focus on the `static_information` as it contains grid information and it can be used to perform a DC loadflow.
It consists of two important data structures:

1. The `solver_config` which contains parameters for the solver that do no change during optimization.
2. The `dynamic_information` which contains grid-related information that might change during optimization.

The latter allows us to understand our grid a bit:

In [ ]:
print("Number of nodes:", static_information.dynamic_information.n_nodes)
print("Number of branches:", static_information.dynamic_information.n_branches)
print("Number of active power node injections:", static_information.dynamic_information.nodal_injections.shape)

Great it seems like our import worked and we have a grid with 72 nodes and 82 branches.

## Compute the loadflow
We can use the static information to compute the loadflow via a function:

In [ ]:
static_information, metrics = run_initial_loadflow(
    static_information,
)

If you are unfamiliar with functional programming, this line may be a bit confusing.
ToOp is based on `Jax` and thus follows a functional programming paradigm.
We pass our `static_information` to the function and it returns an updated version that includes our resulting DC loadflow.
If you read the annotations of the `DynamicInformation` data structure you can see that this loadflow is stored as `unsplit_flow`.

Remember that we aim to perform topology optimization and this initial loadflow represents the first step for doing so. 

Let's check if our flow matches the expected dimensions:

In [ ]:
print("Number of flows:", static_information.dynamic_information.unsplit_flow.shape)
assert static_information.dynamic_information.unsplit_flow.shape == (static_information.dynamic_information.n_timesteps, static_information.dynamic_information.n_branches)

You may wonder why the shape of the flows and the node injections contain a leading 1.
This is due to the solver's functionality to compute the loadflow for a number of timesteps.
You can see the number of timesteps by inspecting `static_information.dynamic_information.n_timesteps`.

Great! We have computed our first loadflow.

Actually, this loadflow has already been computed when calling `load_grid` but for the sake of this introduction, we recomputed it.